<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchainhub langchain_experimental openai streamlit
!pip install google-search-results wikipedia

In [6]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
import os
import langchain
import openai
import streamlit as st

In [23]:
# Download "Chinook" Music Sales DB
import os
import requests
import zipfile
fname = 'chinook.zip'
url = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/' + fname
r = requests.get(url)
open(fname, 'wb').write(r.content)
zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [24]:
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')
# search = SerpAPIWrapper(serpapi_api_key="Your_SearAPI_Key")
db = SQLDatabase.from_uri("sqlite:///chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    #Tool(
    #    name = "Search",
    #    func=search.run,
    #    description="useful for when you need to answer questions about real-time events. You should ask targeted questions"
    #),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to answer questions about a big picture or background of something."
    ),
    Tool(
        name="MusicSales",
        func=db_chain.run,
        description="useful for when you need to answer questions about mucis sales in a store. Should be strickly follow the tables info."
    )
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


In [25]:
# prompt = "How many people live in the US?"
prompt = "What is the best selling song in the music store?"
response = agent_executor.invoke({"input": prompt})["output"]



> Entering new AgentExecutor chain...

Invoking: `MusicSales` with `best selling song`




> Entering new SQLDatabaseChain chain...
best selling song
SQLQuery:SELECT "tracks"."Name", SUM("invoice_items"."Quantity") as "TotalSold"
FROM "tracks"
JOIN "invoice_items" ON "tracks"."TrackId" = "invoice_items"."TrackId"
GROUP BY "tracks"."Name"
ORDER BY "TotalSold" DESC
LIMIT 1
SQLResult: [('The Trooper', 5)]
Answer:The best selling song is 'The Trooper'.
> Finished chain.
The best selling song is 'The Trooper'.The best selling song in the music store is 'The Trooper'.

> Finished chain.


In [26]:
print(response)

The best selling song in the music store is 'The Trooper'.


In [17]:
print(db)